In [1]:
import pandas as pd

In [2]:
# Import both of the input files
vf_df = pd.read_csv('./dc_vf_sample_20220829.csv')
fund_df = pd.read_csv('./dc_fundraising_sample_20220829.csv') 

### Step 1: Transform Categorical Data to Binary Columns

Since the values in the `ethnicity` and `religion` columns can get long, I'll first shorten each possible value in each column to a two or three character code. This will give the resulting dataframe manageable and consistently-sized column names.

(I also try to avoid creating column names with spaces whenever possible, but that's just a personal style thing.) 

In [3]:
religion_map = {
    'Protestant': 'PR',
    'Catholic': 'CA',
    'Christian': 'CR',
    'Jewish': 'JW',
    'Islamic': 'IS',
    'Buddhist': 'BU',
    'Lutheran': 'LU',
    'Eastern Orthodox': 'EO',
    'Hindu': 'HI',
    'Shinto': 'SH',
    'Greek Orthodox': 'GO'
}

ethnicity_map = {
    'Likely African-American': 'LAA',
    'European': 'EUR',
    'East and South Asian': 'ESA',
    'Other': 'OTH',
    'Hispanic and Portuguese': 'HAP'
}

# Apply these mappings to the voter file dataframe
vf_df['religion'] = vf_df['religion'].map(religion_map)
vf_df['ethnicity'] = vf_df['ethnicity'].map(ethnicity_map)

In [4]:
# construct the VF table with binary gender, religion, and ethnicity columns.
vf_bin_df = pd.get_dummies(vf_df, columns=['gender', 'religion', 'ethnicity'])

### Step 2: Aggregate Fundraising Data for Biden and Bernie

In [5]:
# Create a new column on the fundraising dataframe that clearly associates 
# each donation record with the committee it was sent to.
def extract_dest_committee(t):
    # Biden
    if 'C00703975' in t:
        return 'B'
    # Sanders
    elif 'C00696948' in t:
        return 'S'
    else:
        # Unknown
        return 'U'

fund_df['dest_committee'] = fund_df['memo_text_description'].apply(lambda e: extract_dest_committee(e))

In [6]:
# Aggregate the fundraising data by person and committee 
agg_fund_df = pd.pivot_table(fund_df, values=['amount'], index=['lalvoterid'], columns='dest_committee', aggfunc=sum)
agg_fund_df = agg_fund_df.reset_index(drop=False)

# Clean up and flatten the pivot table
agg_fund_df.columns = ['lalvoterid', 'TOTAL_BIDEN', 'TOTAL_SANDERS']
agg_fund_df = agg_fund_df.fillna(0)

### Step 3: Join Aggregated Finance Data to the Voter File

In [7]:
# By doing an outer merge, we guarantee that we won't throw out any of the 10k VF records that we started with. 
joined_vf_df = vf_bin_df.merge(agg_fund_df, how='outer', on='lalvoterid')
# The NaN donation values may be useful for analysis purposes, but setting them to zero creates a cleaner final product. 
joined_vf_df['TOTAL_BIDEN'] = joined_vf_df['TOTAL_BIDEN'].fillna(0)
joined_vf_df['TOTAL_SANDERS'] = joined_vf_df['TOTAL_SANDERS'].fillna(0)

In [9]:
# 448 people in the voter file sample set donated a non-zero amount of money to either the Biden or Sanders campaign.
joined_vf_df[(joined_vf_df['TOTAL_BIDEN']>0) | (joined_vf_df['TOTAL_SANDERS']>0)]

,lalvoterid,zip,age,gender_F,gender_M,religion_BU,religion_CA,religion_CR,religion_EO,religion_GO,...,religion_LU,religion_PR,religion_SH,ethnicity_ESA,ethnicity_EUR,ethnicity_HAP,ethnicity_LAA,ethnicity_OTH,TOTAL_BIDEN,TOTAL_SANDERS
15,LALDC174655674,20015.0,62,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,12.5,0.0
36,LALDC174726358,20002.0,72,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1000.0,0.0
51,LALDC174769518,20007.0,66,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,25.0,0.0
60,LALDC174794645,20011.0,55,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,100.0,0.0
67,LALDC174809805,20008.0,48,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0.0,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,LALDC175062957,20018.0,42,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,76.5,0.0
9875,LALDC486513994,20001.0,33,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,25.0,0.0
9888,LALDC506009333,20016.0,41,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,300.0,0.0
9894,LALDC506030646,20009.0,51,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,25.0,0.0


In [10]:
joined_vf_df.to_csv('./joined_vf_fundraising.csv')